# Import module 

the without '_y' network structure borrow from https://github.com/xudonmao/LSGAN/blob/master/vgg.py   
the with 'y' network structure borrow from https://github.com/xudonmao/LSGAN/blob/master/mcgan.py

In [2]:
import os
import math
import scipy
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Loading MNIST data

In [3]:
os.environ['CUDA_VISIBLE_DEVICES']='1'
mnist = input_data.read_data_sets("data/mnist",one_hot=True)

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


# Helper Functions

In [4]:
def bn(net,scope,is_training):
    return tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                            is_training=is_training, scope=scope)

def conv(net,wscope,bscope,output_depth = 64, receptive_field=[5,5],stride=[2,2]):
    shape = net.get_shape()
    weights = tf.get_variable(wscope, receptive_field+[shape[-1], output_depth],initializer=tf.truncated_normal_initializer(stddev=0.02))
    biases = tf.get_variable(bscope, [output_depth], initializer=tf.constant_initializer(0.0))
    net = tf.nn.conv2d(net, weights, strides=[1]+stride+[1], padding='SAME')
    net = tf.reshape(tf.nn.bias_add(net, biases), net.get_shape())
    return net

def linear(net,wscope,bscope,output_depth):
    shape = net.get_shape()       
    weights = tf.get_variable(wscope, [shape[-1], output_depth], tf.float32,tf.random_normal_initializer(stddev=0.02))
    biases = tf.get_variable(bscope, [output_depth], initializer=tf.constant_initializer(0.0))
    out_logit = tf.matmul(net, weights) + biases
    return out_logit

def deconv(net,wscope,bscope,output_shape,receptive_field=[5,5],stride=[2,2]):
    weights = tf.get_variable(wscope, receptive_field+[output_shape[-1], net.get_shape()[-1]],
                                       initializer=tf.random_normal_initializer(stddev=0.02))
    net = tf.nn.conv2d_transpose(net, weights, output_shape=output_shape, strides=[1]+stride+[1])
    biases = tf.get_variable(bscope, [output_shape[-1]], initializer=tf.constant_initializer(0.0))
    net = tf.reshape(tf.nn.bias_add(net, biases), net.get_shape())
    return net

def get_shape(net):
    return net.get_shape().as_list()

def mse(pred,label):
    return tf.reduce_mean(tf.nn.l2_loss(pred - label)) 

# Discriminator Network

Here,i will write two "discriminator" function, the "discriminator_y" only has more one "y" parameter than "discriminator". Spliting them into two function in order to easily understanding the structure

In [5]:
def discriminator(inputs, batch_size, is_training=True, reuse=False):
    
    with tf.variable_scope("discriminator", reuse=reuse):
        
        '''1st: Conv -> lrelu'''
        net = conv(inputs,'d_wconv1','d_bconv1',64)
        net = tf.nn.leaky_relu(net)

        '''2nd: Conv -> bn -> lrelu'''
        net = conv(net,'d_wconv2','d_bconv2',64*2)
        net = bn(net, scope = 'd_bn2',is_training=is_training)
        net = tf.nn.leaky_relu(net)

        '''3th: Conv -> bn -> lrelu'''
        net = conv(net,'d_wconv3','d_bconv3',64*4)
        net = bn(net, scope = 'd_bn3',is_training=is_training)
        net = tf.nn.leaky_relu(net)

        '''4th: Conv -> bn -> lrelu'''
        net = conv(net,'d_wconv4','d_bconv4',64*8)
        net = bn(net, scope = 'd_bn4',is_training=is_training)
        net = tf.nn.leaky_relu(net)

        net = tf.reshape(net, [batch_size, -1])   

        '''5th: linear '''
        out_logit = linear(net,"d_wlinear5","d_blinear5",1)
        '''6th: sigmoid'''
        out = tf.nn.sigmoid(out_logit)        

        return out, out_logit

In [6]:
def discriminator_y(inputs, y, y_dim, batch_size, is_training=True, reuse=False):
    
    with tf.variable_scope("discriminator_y", reuse=reuse):

        '''0:label onehot -> linear'''
        y_linear = linear(y,'d_wlinear0','d_linear0',256)
        
        yb = tf.reshape(y_linear,shape=[batch_size,1,1,256])
        
        '''1st:conv -> lrelu -> Concat'''
        net = conv(inputs,'d_wconv1','d_bconv1',inputs.get_shape()[-1]+256)
        net = tf.nn.leaky_relu(net)
        net = tf.concat([net, yb*tf.ones(get_shape(net)[:-1]+[get_shape(yb)[-1]])],axis=3) 
        
        '''2nd:conv -> bn -> lrelu -> reshape -> Concat'''
        net = conv(net,'d_wconv2','d_bconv2',64+256)        
        net = bn(net, scope = 'd_bn2',is_training=is_training)
        net = tf.nn.leaky_relu(net)
        net = tf.reshape(net,[batch_size,-1])
        net = tf.concat([net, y_linear],axis=1)

        '''3th:linear -> bn -> lrelu -> Concat'''
        net = linear(net,"d_wlinear3","d_blinear3",1024)        
        net = bn(net, scope = 'd_bn3',is_training=is_training)
        net = tf.nn.leaky_relu(net)
        net = tf.concat([net, y_linear],axis=1)
        
        '''4th: linear'''
        out_logit = linear(net,"d_wlinear4","d_blinear4",1)  
        '''5th: sigmoid'''
        out = tf.nn.sigmoid(out_logit)        

        return out, out_logit

# Generator Network

Here,i will write two "generator" function, the "generator_y" only has more one "y" parameter than "generator". Spliting them into two function in order to easily understanding the structure

In [5]:
def generator( z, batch_size, output_height=28, output_width=28,output_depth=1, is_training=True, reuse=False):
    
    height0,width0 = output_height,output_width
    height2,width2 = math.ceil(float(height0)/2),math.ceil(float(width0)/2)
    height4,width4 = math.ceil(float(height2)/2),math.ceil(float(width2)/2)
    height8,width8 = math.ceil(float(height4)/2),math.ceil(float(width4)/2)
    height16,width16 = math.ceil(float(height8)/2),math.ceil(float(width8)/2)
    
    with tf.variable_scope("generator", reuse=reuse):
        
        '''1st: linear -> reshape -> bn -> relu '''
        net = linear(z,"g_wlinear1","g_blinear1",64*4*height16*width16)
        net = tf.reshape(net,[batch_size,height16,width16,64*4])
        net = bn(net,scope='g_bn1',is_training=is_training)
        net = tf.nn.relu(net)  
    
        '''2nd: deconv -> bn -> relu '''
        output_shape = [batch_size, height8, width8, 64*4]
        net = deconv(net,'g_wdeconv2','g_bdeconv2',output_shape,[3,3],[2,2])
        net = bn(net,scope='g_bn2',is_training=is_training)
        net = tf.nn.relu(net)

        '''2th-extend: deconv -> bn -> relu '''
        output_shape = [batch_size, height8, width8, 64*4]
        net = deconv(net,'g_wdeconv21','g_bdeconv21',output_shape,[3,3],[1,1])
        net = bn(net,scope='g_bn21',is_training=is_training)
        net = tf.nn.relu(net)
        
        '''3th: deconv -> bn -> relu '''
        output_shape = [batch_size, height4, width4, 64*4]
        net = deconv(net,'g_wdeconv3','g_bdeconv3',output_shape,[3,3],[2,2])
        net = bn(net,scope='g_bn3',is_training=is_training)
        net = tf.nn.relu(net)

        '''3th-extend: deconv -> bn -> relu '''
        output_shape = [batch_size, height4, width4, 64*4]
        net = deconv(net,'g_wdeconv31','g_bdeconv31',output_shape,[3,3],[1,1])
        net = bn(net,scope='g_bn31',is_training=is_training)
        net = tf.nn.relu(net)
        
        '''4th: deconv -> bn -> relu '''
        output_shape = [batch_size, height2, width2, 64*2]
        net = deconv(net,'g_wdeconv4','g_bdeconv4',output_shape,[3,3])
        net = bn(net,scope='g_bn4',is_training=is_training)
        net = tf.nn.relu(net)
        
        '''5th: deconv -> bn -> relu '''
        output_shape = [batch_size, height0, width0, 64*1]
        net = deconv(net,'g_wdeconv5','g_bdeconv5',output_shape,[3,3])
        net = bn(net,scope='g_bn5',is_training=is_training)
        net = tf.nn.relu(net)

        '''6th: deconv -> tanh '''
        output_shape = [batch_size, height0, width0, output_depth]
        net = deconv(net,'g_wdeconv6','g_bdeconv6',output_shape,[3,3],[1,1])
        out = tf.nn.sigmoid(net)

        return out

In [7]:
def generator_y( z, y, y_dim, batch_size, output_height, output_width,output_depth, is_training=True, reuse=False):
    
    height0,width0 = output_height,output_width
    height2,width2 = math.ceil(float(height0)/2),math.ceil(float(width0)/2)
    height4,width4 = math.ceil(float(height2)/2),math.ceil(float(width2)/2)
    
    with tf.variable_scope("generator_y", reuse=reuse):
        
        '''0:label onehot -> linear'''
        y_linear = linear(y,'g_wlinear0','g_linear0',256)
        
        yb = tf.reshape(y_linear,[batch_size,1,1,256])
        
        '''1st:Concat -> linear -> bn -> relu '''
        z = tf.concat([z,y_linear],axis=1)
        net = linear(z,"g_wlinear1","g_blinear1",128*height4*width4)
        net = bn(net,scope='g_bn1',is_training=is_training)
        net = tf.nn.relu(net)  
    
        net = tf.reshape(net,[batch_size,height4,width4,128])
        
        '''2nd: deconv -> bn -> relu '''
        output_shape = [batch_size, height2, width2, 128]
        net = deconv(net,'g_wdeconv2','g_bdeconv2',output_shape)     
        net = bn(net,scope='g_bn2',is_training=is_training)
        net = tf.nn.relu(net)

        '''3th: deconv -> sigmoid '''
        output_shape = [batch_size, height0, width0, output_depth]
        net = deconv(net,'g_wdeconv3','g_bdeconv3',output_shape)
        out = tf.nn.sigmoid(net)

        return out

# set the global parameters.

change the value of "y_flag" to choose the different network

In [8]:
# some parameters
image_dims = [28, 28, 1]
batch_size = 64
z_dim = 1024
y_dim = 10
learning_rate = 0.0002 #0.001
beta1 = 0.5
y_flag = True
output_height,output_width,output_depth = [28,28,1]
""" Graph Input """
# images
inputs = tf.placeholder(tf.float32, [batch_size] + image_dims, name='real_images')
#labels
y = tf.placeholder(tf.float32, [batch_size,y_dim], name='y')
# noises
z = tf.placeholder(tf.float32, [batch_size, z_dim], name='z')

# Loss Function 

the "mse" loss function borrowed from "loss_l2" in https://github.com/xudonmao/LSGAN/blob/master/vgg.py

In [9]:
""" Loss Function """

if not y_flag:
    # output of D for real images
    D_real, D_real_logits = discriminator(inputs, batch_size, is_training=True, reuse=False)
    # output of D for fake images
    G = generator(z, batch_size, is_training=True, reuse=False)
    D_fake, D_fake_logits = discriminator(G, batch_size, is_training=True, reuse=True)
else:  
    D_real, D_real_logits = discriminator_y(inputs, y, y_dim, batch_size, is_training=True, reuse=False)
    G = generator_y(z, y, y_dim, batch_size, output_height, output_width,output_depth, is_training=True, reuse=False)
    D_fake, D_fake_logits = discriminator_y(G, y, y_dim, batch_size, is_training=True, reuse=True)


# get loss for discriminator
d_loss_real = mse(D_real_logits,tf.ones_like(D_real_logits))
d_loss_fake = mse(D_fake_logits,tf.zeros_like(D_fake_logits))
d_loss = d_loss_real + d_loss_fake

# get loss for generator
g_loss = mse(D_fake_logits,tf.ones_like(D_fake_logits))


split the generator parameters and discriminator parameters into two list, then define how to train the two subnetwork and get the fake image for testing.

In [10]:
""" Training """
# divide trainable variables into a group for D and a group for G
t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'd_' in var.name]
g_vars = [var for var in t_vars if 'g_' in var.name]
# optimizers
d_optim = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(d_loss, var_list=d_vars)
g_optim = tf.train.AdamOptimizer(learning_rate*5, beta1=beta1).minimize(g_loss, var_list=g_vars)


"""" Testing """
# for test
if not y_flag:
    fake_images = generator(z,batch_size, is_training=False, reuse=True)
else:
    fake_images = generator_y(z, y, y_dim, batch_size, output_height, output_width,output_depth, is_training=False, reuse=True)
# graph inputs for visualize training results
sample_z = np.random.uniform(-1, 1, size=(batch_size , z_dim))

test_labels_onehot = np.zeros([batch_size, y_dim],dtype = np.float32)
test_labels_onehot[np.arange(batch_size), np.arange(batch_size)%int(y_dim)] = 1.0

# Training a GAN

In [ ]:
start_epoch = 0
start_batch_id = 0


num_steps = 6000
# loop for epoch
start_time = time.time()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step_ind in range(num_steps):
    
    '''get the real data'''
    real_image_batch = mnist.train.next_batch(batch_size)
    batch_images,batch_labels = real_image_batch
    batch_images = batch_images.reshape([batch_size,28,28,1]).astype(np.float32)
    batch_labels = real_image_batch[1].astype(np.float32)
    '''get the noise data'''
    batch_z = np.random.uniform(-1, 1, [batch_size, z_dim]).astype(np.float32)

    if not y_flag:
        # update D network
        _ , D_loss = sess.run([d_optim, d_loss], feed_dict={inputs: batch_images, z: batch_z})
        # update G network
        _, G_loss = sess.run([g_optim,g_loss], feed_dict={z: batch_z})
        _, G_loss = sess.run([g_optim,g_loss], feed_dict={z: batch_z})
        _, G_loss = sess.run([g_optim,g_loss], feed_dict={z: batch_z})
    else:
        _ , D_loss = sess.run([d_optim, d_loss], feed_dict={inputs: batch_images, y:batch_labels, z: batch_z})
        _, G_loss = sess.run([g_optim,g_loss], feed_dict={z: batch_z,y:batch_labels})
        _, G_loss = sess.run([g_optim,g_loss], feed_dict={z: batch_z,y:batch_labels})
        
        
    # display training status
    print("Step: [%d] d_loss: %.8f, g_loss: %.8f" % (step_ind, D_loss, G_loss) )

    # save training results for every 300 steps
    if np.mod(step_ind, 300) == 0:
        if not y_flag:
            samples = sess.run(fake_images, feed_dict={z: sample_z})
        else:
            samples = sess.run(fake_images, feed_dict={z: sample_z,y:test_labels_onehot})
        # put the "batch_size" images into one big canvas
        row = col = int(np.sqrt(batch_size))
        img = np.zeros( [row*28, col*28] )
        for i in range(row):
            for j in range(col):
                img[i*28:(i+1)*28,j*28:(j+1)*28] = samples[i*col+j, :, :, :].squeeze()
        #save the result      
        scipy.misc.imsave('multi_{}.jpg'.format(step_ind),img)

Step: [0] d_loss: 59.80548477, g_loss: 198.83297729
Step: [1] d_loss: 704.10308838, g_loss: 31.14851761
Step: [2] d_loss: 96.46445465, g_loss: 30.23890495
Step: [3] d_loss: 80.44414520, g_loss: 25.73360443
Step: [4] d_loss: 58.87501526, g_loss: 22.19931221
Step: [5] d_loss: 33.26681519, g_loss: 19.41390991
Step: [6] d_loss: 33.03694153, g_loss: 17.47875977
Step: [7] d_loss: 29.14501381, g_loss: 16.15901756
Step: [8] d_loss: 24.76525497, g_loss: 15.32536221
Step: [9] d_loss: 23.73504448, g_loss: 13.89806366
Step: [10] d_loss: 19.98196602, g_loss: 12.84476376
Step: [11] d_loss: 23.16379547, g_loss: 12.90688992
Step: [12] d_loss: 27.85366821, g_loss: 14.37609482
Step: [13] d_loss: 37.55335617, g_loss: 13.52400398
Step: [14] d_loss: 33.28873825, g_loss: 12.26737309
Step: [15] d_loss: 16.50245094, g_loss: 12.29934502
Step: [16] d_loss: 15.07438087, g_loss: 11.79363632
Step: [17] d_loss: 15.42598629, g_loss: 11.70466423
Step: [18] d_loss: 17.17868996, g_loss: 11.50112152
Step: [19] d_loss: 1

Step: [160] d_loss: 16.22226334, g_loss: 7.80475855
Step: [161] d_loss: 16.44029999, g_loss: 8.43337727
Step: [162] d_loss: 14.57096100, g_loss: 7.84488487
Step: [163] d_loss: 15.20720387, g_loss: 8.55621147
Step: [164] d_loss: 16.44831276, g_loss: 8.02939606
Step: [165] d_loss: 15.50082397, g_loss: 7.82508564
Step: [166] d_loss: 14.06492805, g_loss: 8.36922264
Step: [167] d_loss: 15.02459812, g_loss: 8.32875919
Step: [168] d_loss: 14.34440804, g_loss: 7.71554852
Step: [169] d_loss: 14.45543861, g_loss: 8.29036045
Step: [170] d_loss: 17.37260437, g_loss: 9.51391315
Step: [171] d_loss: 18.63050270, g_loss: 8.52787399
Step: [172] d_loss: 15.46462822, g_loss: 8.27594471
Step: [173] d_loss: 15.45125771, g_loss: 7.83220863
Step: [174] d_loss: 14.31082344, g_loss: 8.05735016
Step: [175] d_loss: 14.90434837, g_loss: 7.57246876
Step: [176] d_loss: 14.96989822, g_loss: 8.15033436
Step: [177] d_loss: 16.94496155, g_loss: 8.56702995
Step: [178] d_loss: 17.08020782, g_loss: 8.06845379
Step: [179] 

Step: [318] d_loss: 14.25713348, g_loss: 7.45875263
Step: [319] d_loss: 14.13189793, g_loss: 7.88650370
Step: [320] d_loss: 14.76621819, g_loss: 7.28332901
Step: [321] d_loss: 14.39624786, g_loss: 8.62301826
Step: [322] d_loss: 15.06126499, g_loss: 8.25271034
Step: [323] d_loss: 14.40639591, g_loss: 8.00706577
Step: [324] d_loss: 14.48849678, g_loss: 7.99848795
Step: [325] d_loss: 16.44473267, g_loss: 8.70788670
Step: [326] d_loss: 18.27835464, g_loss: 10.03105736
Step: [327] d_loss: 20.91108894, g_loss: 10.37660027
Step: [328] d_loss: 20.30762863, g_loss: 9.84776402
Step: [329] d_loss: 17.08173561, g_loss: 7.88208103
Step: [330] d_loss: 16.32494736, g_loss: 7.26143026
Step: [331] d_loss: 16.42480278, g_loss: 7.62334442
Step: [332] d_loss: 15.71599293, g_loss: 6.98663568
Step: [333] d_loss: 14.19481564, g_loss: 8.19939995
Step: [334] d_loss: 14.78369713, g_loss: 7.17254591
Step: [335] d_loss: 15.58484840, g_loss: 6.91064739
Step: [336] d_loss: 14.89207268, g_loss: 7.00489187
Step: [337

Step: [476] d_loss: 15.30159378, g_loss: 6.78446674
Step: [477] d_loss: 15.94261360, g_loss: 7.42767525
Step: [478] d_loss: 15.56146622, g_loss: 7.56897974
Step: [479] d_loss: 16.55422974, g_loss: 5.93714142
Step: [480] d_loss: 15.48546219, g_loss: 6.81594610
Step: [481] d_loss: 15.80926609, g_loss: 7.54415274
Step: [482] d_loss: 15.55617142, g_loss: 6.85685158
Step: [483] d_loss: 15.25650120, g_loss: 7.67067003
Step: [484] d_loss: 15.72834969, g_loss: 7.42464018
Step: [485] d_loss: 15.96080971, g_loss: 8.03066826
Step: [486] d_loss: 17.03796196, g_loss: 7.73152065
Step: [487] d_loss: 15.82363987, g_loss: 7.82967138
Step: [488] d_loss: 16.13913345, g_loss: 7.64351797
Step: [489] d_loss: 16.16292381, g_loss: 7.57613945
Step: [490] d_loss: 15.55464077, g_loss: 7.76876926
Step: [491] d_loss: 15.53186226, g_loss: 7.72811937
Step: [492] d_loss: 15.29586792, g_loss: 7.13885880
Step: [493] d_loss: 15.25792885, g_loss: 7.32140732
Step: [494] d_loss: 14.68978310, g_loss: 7.15515804
Step: [495] 

Step: [634] d_loss: 15.50947094, g_loss: 6.99222088
Step: [635] d_loss: 16.56053543, g_loss: 6.90817976
Step: [636] d_loss: 15.91994381, g_loss: 7.05746222
Step: [637] d_loss: 15.40920544, g_loss: 7.20001793
Step: [638] d_loss: 15.69064713, g_loss: 7.39384651
Step: [639] d_loss: 15.77560234, g_loss: 7.47710705
Step: [640] d_loss: 16.33490753, g_loss: 7.70150137
Step: [641] d_loss: 16.02876472, g_loss: 7.05621338
Step: [642] d_loss: 15.37630463, g_loss: 6.91829586
Step: [643] d_loss: 15.09439087, g_loss: 7.72519732
Step: [644] d_loss: 15.92959976, g_loss: 7.14422321
Step: [645] d_loss: 14.87419891, g_loss: 6.92172718
Step: [646] d_loss: 15.60677719, g_loss: 7.15760565
Step: [647] d_loss: 15.15883827, g_loss: 7.10189724
Step: [648] d_loss: 14.81790924, g_loss: 7.54995251
Step: [649] d_loss: 15.56762886, g_loss: 6.92688465
Step: [650] d_loss: 15.08373070, g_loss: 7.28453875
Step: [651] d_loss: 14.95661640, g_loss: 7.16095734
Step: [652] d_loss: 15.56930256, g_loss: 6.95965004
Step: [653] 

...

Step: [2656] d_loss: 15.83990669, g_loss: 8.11775970
Step: [2657] d_loss: 16.09969711, g_loss: 8.32792187
Step: [2658] d_loss: 15.79921436, g_loss: 8.56488705
Step: [2659] d_loss: 16.20047379, g_loss: 8.06417084
Step: [2660] d_loss: 14.98905182, g_loss: 8.42740250
Step: [2661] d_loss: 15.31031990, g_loss: 8.25246620
Step: [2662] d_loss: 14.90103722, g_loss: 9.06630707
Step: [2663] d_loss: 15.74878407, g_loss: 8.02891541
Step: [2664] d_loss: 15.17765999, g_loss: 8.11551285
Step: [2665] d_loss: 15.50625992, g_loss: 8.28870201
Step: [2666] d_loss: 15.61964035, g_loss: 8.13429260
Step: [2667] d_loss: 13.81914139, g_loss: 8.69431305
Step: [2668] d_loss: 14.93066883, g_loss: 8.19602394
Step: [2669] d_loss: 15.32052422, g_loss: 8.10107231
Step: [2670] d_loss: 15.02687836, g_loss: 8.45867825
Step: [2671] d_loss: 15.57956314, g_loss: 7.76378727
Step: [2672] d_loss: 15.44180393, g_loss: 8.05081749
Step: [2673] d_loss: 15.62375546, g_loss: 8.41256618
Step: [2674] d_loss: 14.87035942, g_loss: 8.05

Step: [2811] d_loss: 14.94493103, g_loss: 8.03144360
Step: [2812] d_loss: 14.37222004, g_loss: 8.46528912
Step: [2813] d_loss: 15.40335083, g_loss: 8.28165340
Step: [2814] d_loss: 15.42823887, g_loss: 7.73121071
Step: [2815] d_loss: 15.36247158, g_loss: 7.46201324
Step: [2816] d_loss: 15.35813808, g_loss: 8.06425190
Step: [2817] d_loss: 14.97722340, g_loss: 8.04838371
Step: [2818] d_loss: 14.70781898, g_loss: 8.18609047
Step: [2819] d_loss: 15.89796448, g_loss: 8.08761215
Step: [2820] d_loss: 15.70787048, g_loss: 8.00870705
Step: [2821] d_loss: 16.15189743, g_loss: 7.83022118
Step: [2822] d_loss: 15.12285519, g_loss: 8.27475643
Step: [2823] d_loss: 15.16548347, g_loss: 7.80832338
Step: [2824] d_loss: 15.23566628, g_loss: 8.10664558
Step: [2825] d_loss: 15.46198177, g_loss: 7.65920687
Step: [2826] d_loss: 15.46007538, g_loss: 7.78274441
Step: [2827] d_loss: 14.93865108, g_loss: 8.71816635
Step: [2828] d_loss: 14.99135590, g_loss: 7.64491892
Step: [2829] d_loss: 14.55924416, g_loss: 8.19

Step: [2966] d_loss: 14.79739571, g_loss: 7.85773373
Step: [2967] d_loss: 14.71571350, g_loss: 7.93587780
Step: [2968] d_loss: 15.21725273, g_loss: 7.73789120
Step: [2969] d_loss: 15.48205090, g_loss: 8.01600933
Step: [2970] d_loss: 15.00143623, g_loss: 8.51794720
Step: [2971] d_loss: 15.16417122, g_loss: 8.73325348
Step: [2972] d_loss: 15.42468071, g_loss: 7.96952724
Step: [2973] d_loss: 15.95949268, g_loss: 8.17978954
Step: [2974] d_loss: 15.38220882, g_loss: 7.96021843
Step: [2975] d_loss: 15.22832298, g_loss: 7.89223433
Step: [2976] d_loss: 15.01269531, g_loss: 7.76074743
Step: [2977] d_loss: 14.85054111, g_loss: 7.75640583
Step: [2978] d_loss: 16.03644180, g_loss: 7.96964645
Step: [2979] d_loss: 15.15337276, g_loss: 7.98992825
Step: [2980] d_loss: 14.93971062, g_loss: 8.95292759
Step: [2981] d_loss: 16.11670685, g_loss: 8.59814930
Step: [2982] d_loss: 15.94100666, g_loss: 8.53676224
Step: [2983] d_loss: 15.17739868, g_loss: 8.10388088
Step: [2984] d_loss: 14.82746792, g_loss: 8.58

Step: [3121] d_loss: 15.13109398, g_loss: 8.18837738
Step: [3122] d_loss: 15.16705513, g_loss: 7.83666468
Step: [3123] d_loss: 14.80226135, g_loss: 8.09801960
Step: [3124] d_loss: 14.87146187, g_loss: 8.29040527
Step: [3125] d_loss: 15.09896564, g_loss: 8.72513008
Step: [3126] d_loss: 15.21020126, g_loss: 8.40773201
Step: [3127] d_loss: 14.72792053, g_loss: 8.07328224
Step: [3128] d_loss: 14.75482368, g_loss: 8.29771900
Step: [3129] d_loss: 15.19664764, g_loss: 9.05560303
Step: [3130] d_loss: 16.10687637, g_loss: 8.26265621
Step: [3131] d_loss: 14.94718361, g_loss: 8.14743519
Step: [3132] d_loss: 14.93882179, g_loss: 8.43783092
Step: [3133] d_loss: 14.76722813, g_loss: 8.52234459
Step: [3134] d_loss: 15.54276848, g_loss: 8.40721512
Step: [3135] d_loss: 15.02667618, g_loss: 7.90543365
Step: [3136] d_loss: 15.30680275, g_loss: 8.02291489
Step: [3137] d_loss: 14.07469654, g_loss: 8.42293930
Step: [3138] d_loss: 14.55643272, g_loss: 8.46585178
Step: [3139] d_loss: 14.64643860, g_loss: 8.67

Step: [3276] d_loss: 15.46169281, g_loss: 7.75211716
Step: [3277] d_loss: 15.42194748, g_loss: 8.04976654
Step: [3278] d_loss: 15.21266556, g_loss: 7.72156477
Step: [3279] d_loss: 15.22696400, g_loss: 8.46161079
Step: [3280] d_loss: 15.74856472, g_loss: 8.14811897
Step: [3281] d_loss: 15.05677986, g_loss: 7.73296547
Step: [3282] d_loss: 14.66836548, g_loss: 8.35695839
Step: [3283] d_loss: 15.60420799, g_loss: 7.37376595
Step: [3284] d_loss: 15.64185715, g_loss: 7.35850573
Step: [3285] d_loss: 14.32859993, g_loss: 8.79332733
Step: [3286] d_loss: 15.22418976, g_loss: 8.29764175
Step: [3287] d_loss: 15.87106228, g_loss: 8.04516029
Step: [3288] d_loss: 15.16284180, g_loss: 8.25758648
Step: [3289] d_loss: 15.11781120, g_loss: 7.87896919
Step: [3290] d_loss: 14.88245010, g_loss: 7.69530582
Step: [3291] d_loss: 15.40637589, g_loss: 7.97959518
Step: [3292] d_loss: 15.18181419, g_loss: 8.63588905
Step: [3293] d_loss: 14.78735924, g_loss: 8.24862576
Step: [3294] d_loss: 14.73544788, g_loss: 8.42